# Web Scraping, Second Pass

Here, we will try to unpickle the list from actor_scrape.ipynb and add the content of more pages to it.

In [2]:
import pickle
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [3]:
with open('top50.pkl', 'r') as picklefile:
    my_old_array = pickle.load(picklefile)

In [4]:
page1text = my_old_array[0]
actors = my_old_array[1]

In [5]:
top_soup = bs(page1text)
print top_soup.prettify()

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   Box Office Mojo - People Index
  </title>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = 'http://s.amazon-adsystem.com/iu3?',
                params_sis3 = [
                    "d=boxofficemojo.com",
                    "cb=" + cacheBust
                ];

            (document.getElementById('sis_pixel_sitewide')).src = url_sis3 + params_sis3.join('&

In [6]:
link_suffixes = []
for i in range(2, 15):
    link_suffixes.append(top_soup.find_all(text=re.compile('\#'))[i].parent.get('href'))
link_suffixes

['/people/?view=Actor&pagenum=2&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=3&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=4&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=5&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=6&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=7&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=8&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=9&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=10&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=11&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=12&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=13&sort=sumgross&order=DESC&&p=.htm',
 '/people/?view=Actor&pagenum=14&sort=sumgross&order=DESC&&p=.htm']

In [7]:
link_prefix = 'http://boxofficemojo.com'
pages = []
for link in link_suffixes:
    pages.append(requests.get(link_prefix + link))

In [8]:
map(lambda x: x.status_code, pages)

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

In [9]:
page_texts = map(lambda x: bs(x.text), pages)

In [10]:
print page_texts[0].prettify()

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   Box Office Mojo - People Index
  </title>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = 'http://s.amazon-adsystem.com/iu3?',
                params_sis3 = [
                    "d=boxofficemojo.com",
                    "cb=" + cacheBust
                ];

            (document.getElementById('sis_pixel_sitewide')).src = url_sis3 + params_sis3.join('&

In [11]:
def strip_dollar(s):
    if s[0] == '$':
        if s[-1] == 'k':
            return float(s[1:-1].replace(',', '')) / 1000
        return float(s[1:].replace(',', ''))
    return s

def get_data(soup):
    rows = []
    my_table = soup.find_all('table')[3].contents[1:]
    for i in range(len(my_table)):
        if my_table[i] == u'\n':
            continue
        row = []
        for cell in my_table[i].find_all('td'):
            for string in cell.strings:
                row.append(strip_dollar(str(string)))
        rows.append(row)
    return rows
len(page_texts)

13

In [12]:
for page in page_texts:
    rows = get_data(page)
    new_actors = pd.DataFrame(zip(*zip(*rows)[1:]), columns = ['actor', 'total_gross', 'num_movies', 'av_gross', 'highest_movie', 'movie_gross'])
    actors = actors.append(new_actors, ignore_index = True)

actors = actors.drop_duplicates()
actors.sort(columns = "av_gross", inplace = True, ascending = False)
actors

,actor,total_gross,num_movies,av_gross,highest_movie,movie_gross
247,Jackson Rathbone,1302.5000,5,260.5000,The Twilight Saga: Eclipse,300.5000
957,Rosie Huntington-Whiteley,506.0000,2,253.0000,Transformers 3,352.4000
417,Lenny Kravitz,996.9000,4,249.2000,Catching Fire,424.7000
443,Irrfan Khan,918.5000,4,229.6000,Jurassic World,650.5000
493,Evangeline Lilly,777.2000,4,194.3000,The Hobbit 2,258.4000
0,Emma Watson,2681.8000,14,191.6000,Harry Potter / Deathly Hallows (P2),381.0000
1,Daniel Radcliffe,2449.2000,13,188.4000,Harry Potter / Deathly Hallows (P2),381.0000
202,Taylor Lautner,1482.2000,8,185.3000,The Twilight Saga: Eclipse,300.5000
2,Rupert Grint,2390.5000,13,183.9000,Harry Potter / Deathly Hallows (P2),381.0000
1022,Karen Gillan,360.9000,2,180.4000,Guardians of the Galaxy,333.2000


Well that's disappointing. I've hardly heard of anyone in the top 10!!

In [13]:
with open('top743.pkl', 'w') as picklefile:
    pickle.dump(actors, picklefile)

In [3]:
with open('top743.pkl', 'r') as f:
    actors = pickle.load(f)

In [26]:
actors['num_movies'] = a.astype('float64')
actors[actors.num_movies >= 20].head(100)

,actor,total_gross,num_movies,av_gross,highest_movie,movie_gross
4,Will Smith,2814.3,22,127.9,Independence Day,306.2
5,Bradley Cooper,2487.6,23,108.2,American Sniper,350.1
59,Shia LaBeouf,2128.6,20,106.4,Transformers 2,402.1
64,Jeremy Renner,2090.7,20,104.5,The Avengers,623.4
6,Steve Carell,2291.6,22,104.2,Despicable Me 2,368.1
7,Ian McKellen,3150.3,31,101.6,Return of the King,377.8
8,Tom Hanks,4264.2,42,101.5,Toy Story 3,415.0
9,Eddie Murphy,3810.4,38,100.3,Shrek 2,441.2
10,Tom Cruise,3585.7,36,99.6,War of the Worlds,234.3
11,Harrison Ford,3935.1,40,98.4,Star Wars,461.0


In [27]:
with open('top100prolific.pkl', 'w') as f:
    pickle.dump(actors[actors.num_movies >= 20].head(100), f)